In [1]:
# importing libraries 
from bs4 import BeautifulSoup 
import requests 
import csv

def scrap(URL): 
    # specifying user agent, You can use other user agents 
    # available on the internet 
    HEADERS = ({'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36', 
                                'Accept-Language': 'en-US, en;q=0.5'}) 
    # Making the HTTP Request 
    webpage = requests.get(URL, headers=HEADERS) 

    # Creating the Soup Object containing all data 
    soup = BeautifulSoup(webpage.content, "html.parser") 
    records=[]
    results=soup.find_all('div',attrs={'data-component-type':'s-search-result'})
    
    for item in results :
      record = extract_record(item)
      if record:
        records.append(record)
    return records

def extract_record(item):

   # retreiving product title 
    try: 
      title = item.find("span",attrs={"class": 'a-size-medium a-color-base a-text-normal'}) 
      title = title.string.strip().replace(',', '') 
     
    except AttributeError: 
      title = "NA"
   # retreiving price 
    try: 
        price = item.find("span", attrs={'class': 'a-offscreen'})
        price=price.string.strip().replace(',', '') 
   
    except AttributeError: 
        price = "NA"
    # retreiving product rating 
    try: 
        rating = item.find("span", attrs={'class': 'a-icon-alt'})
        rating=rating.string.strip().replace(',', '') 

    except: 
         rating = "NA"
   
    #retrieving review count
    try: 
        review_count = item.find("span", attrs={'class': 'a-size-base'})
        review_count=review_count.text 

    except AttributeError: 
        review_count = "NA"
  
    # retrieve delivery status 
    try: 
        delivery = item.find("span", attrs={'aria-label': 'FREE Delivery by Amazon'}) 
        delivery = delivery.find("span").string.strip().replace(',', '') 

    except AttributeError: 
        delivery = "No free delivery"
    
    result=(title,price,rating,review_count,delivery)
    return result
    
results=[]
for page in range(1,21):
  url='https://www.amazon.in/s?k=shoes&i=shoes&rh=n%3A1983518031%2Cp_36%3A100000-250000&dc&page={}&qid=1631962645&rnid=4516629031&ref=sr_nr_p_36_1'.format(page)
  records=scrap(url)
  results=results+records
with open("data.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerow(['Title','Price','Ratings','Review Count','Free Ship'])
      writer.writerows(results)